In [1]:
!pip install redis
!pip install line-bot-sdk

In [2]:
"""

    1.針對跟redis的連線
    

"""

import redis

#製作redis連線
redis = redis.Redis(
    #redis container的host name
    host='redis',
    port=6379,
    #預設沒密碼
    password=None,
    #給格式
    charset="utf-8",
    #要解碼不然回傳的資料前面會多一個b
    decode_responses=True)


In [3]:
#redis.hmset("test",{"message":"88"})
redis.sadd("my_set", "87", "88", "89")

0

In [4]:
redis.scard("my_set")
redis.smembers("my_set")
"87" in redis.smembers("my_set")

True

In [2]:
!pip install python-logstash

  Running setup.py bdist_wheel for python-logstash ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/ae/be/2d/626d5c1fa358aada2de09a07c2318848ad35afada0f9651665
Successfully built python-logstash


In [4]:
import logging
import logstash
import sys

host = 'logstash'

test_logger = logging.getLogger('python-logstash-logger')
test_logger.setLevel(logging.INFO)

# UDP
# test_logger.addHandler(logstash.LogstashHandler(host, 12201, version=1))

# TCP
test_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

# test_logger.error('python-logstash: test logstash error message.')
# test_logger.info('python-logstash: test logstash info message.')
# test_logger.warning('python-logstash: test logstash warning message.')

# add extra field to logstash message
extra = {
    'test_string': 'python version: ' + repr(sys.version_info),
    'test_boolean': True,
    'test_dict': {'a': 1, 'b': 'c'},
    'test_float': 1.23,
    'test_integer': 123,
    'test_list': [1, 2, '3'],
}
test_logger.error('python-logstash: test extra fields', extra=extra)
test_logger.handlers.clear()
print('done,please see kibana')

done,please see kibana


In [6]:
import redis
from hashlib import md5


class SimpleHash(object):
    def __init__(self, cap, seed):
        self.cap = cap
        self.seed = seed

    def hash(self, value):
        ret = 0
        for i in range(len(value)):
            ret += self.seed * ret + ord(value[i])
        return (self.cap - 1) & ret


class BloomFilter(object):
    def __init__(self, host='redis', port=6379, db=0, blockNum=1, key='bloomfilter'):
        """
        :param host: the host of Redis
        :param port: the port of Redis
        :param db: witch db in Redis
        :param blockNum: one blockNum for about 90,000,000; if you have more strings for filtering, increase it.
        :param key: the key's name in Redis
        """
        self.server = redis.Redis(host=host, port=port, db=db)
        self.bit_size = 1 << 31  # Redis的String类型最大容量为512M，现使用256M
        self.seeds = [5, 7, 11, 13, 31, 37, 61]
        self.key = key
        self.blockNum = blockNum
        self.hashfunc = []
        for seed in self.seeds:
            self.hashfunc.append(SimpleHash(self.bit_size, seed))

    def isContains(self, str_input):
        if not str_input:
            return False
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        ret = True
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            ret = ret & self.server.getbit(name, loc)
        return ret

    def insert(self, str_input):
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            self.server.setbit(name, loc, 1)

In [8]:
import redis
from hashlib import md5

            
bf = BloomFilter()
url = "https://tw.news.appledaily.com/politics/realtime/20181125/1473059/"
url = url.encode('utf-8')
if bf.isContains(url):   # 判断字符串是否存在
    print('exists!')
else:
    print('not exists!')
    bf.insert(url)

exists!


In [17]:
import time

print(time.asctime())

Sat Dec  1 09:14:16 2018


In [9]:
import datetime
today = datetime.date.today()
today

datetime.date(2018, 11, 29)